In [126]:
import pandas as pd
import numpy as np
import plotly.express as px
from itertools import product

In [127]:
planning = pd.read_excel('../dataset/a6_booster_delivery_planning.xlsx')
tracks = pd.read_excel('../dataset/ariane_tracks_booster_container_2.0.xlsx')

tracks['Entry'] = pd.to_datetime(tracks['Entry'], format='%d.%m.%Y')
tracks['Exit'] = pd.to_datetime(tracks['Exit'], format='%d.%m.%Y')

In [128]:
sorted_tracks = tracks.sort_values(by=['Container','Entry'])

In [129]:
dates = pd.date_range(start=min(sorted_tracks['Entry']), end=max(sorted_tracks['Entry'])).to_list()
containers = list(sorted_tracks['Container'].unique())

In [130]:
df = pd.DataFrame(list(product(dates,containers)), columns=['Date', 'Container'])

In [131]:
df

,Date,Container
0,2020-12-09,SN1
1,2020-12-09,SN2
2,2020-12-09,SN3
3,2020-12-09,SN4
4,2020-12-09,SN5
...,...,...
9923,2024-05-02,SN4
9924,2024-05-02,SN5
9925,2024-05-02,SN6
9926,2024-05-02,SN7


In [132]:
crossed_data = (df
                .merge(sorted_tracks[['Container', 'Zone', 'Entry', 'Exit']], how = 'cross')
                .query('Entry <= Date <= Exit')
)

In [133]:
crossed_data = crossed_data[crossed_data['Container_x'] == crossed_data['Container_y']].rename(columns={'Container_x':'Container'}).drop(columns='Container_y').reset_index(drop=True)

In [134]:
crossed_data

,Date,Container,Zone,Entry,Exit
0,2020-12-09,SN5,A,2020-12-09,2020-12-24
1,2020-12-09,SN6,A,2020-12-09,2020-12-24
2,2020-12-10,SN5,A,2020-12-09,2020-12-24
3,2020-12-10,SN6,A,2020-12-09,2020-12-24
4,2020-12-11,SN5,A,2020-12-09,2020-12-24
...,...,...,...,...,...
2377,2024-05-01,SN6,C,2024-05-01,2024-05-02
2378,2024-05-02,SN3,C,2024-05-01,2024-05-02
2379,2024-05-02,SN3,B,2024-05-02,2024-05-07
2380,2024-05-02,SN6,C,2024-05-01,2024-05-02


#### Add the coordinates to each of the zones

In [135]:
# crossed_data.to_csv('../dataset/crossed_data.csv', index=False)

In [136]:
"""
import matplotlib.pyplot as plt


# Coordinates of the regular pentagon vertices
vertices = [
    (1, 0),            # Vertex 1
    (0.309, 0.951),    # Vertex 2
    (-0.809, 0.588),   # Vertex 3
    (-0.809, -0.588),  # Vertex 4
    (0.309, -0.951)    # Vertex 5
]

# Extract x and y coordinates
y_coords = [vertex[0] for vertex in vertices]
x_coords = [vertex[1] for vertex in vertices]

# Plot the regular pentagon
plt.figure(figsize=(10, 10))
plt.plot(x_coords + [x_coords[0]], y_coords + [y_coords[0]], 'b-')  # Closing the loop
plt.scatter(x_coords[0], y_coords[0], s=200, color='dodgerblue',alpha=0.5, label='Area A')
plt.scatter(x_coords[1], y_coords[1], s=200, color='indigo',alpha=0.5, label='Area B')
plt.scatter(x_coords[2], y_coords[2], s=200, color='blue',alpha=0.5, label='Area C')
plt.scatter(x_coords[3], y_coords[3], s=200, color='limegreen',alpha=0.5, label='Area D')
plt.scatter(x_coords[4], y_coords[4], s=200, color='black',alpha=0.5, label='Area E')

plt.title('Supply Chain')
#plt.xlabel('x')
#plt.ylabel('y')
plt.xticks([])
plt.xlim(-2, 2)
plt.yticks([])

plt.grid(True)
plt.axis('equal')
plt.savefig("foo2.png")
plt.show()
"""

'\nimport matplotlib.pyplot as plt\n\n\n# Coordinates of the regular pentagon vertices\nvertices = [\n    (1, 0),            # Vertex 1\n    (0.309, 0.951),    # Vertex 2\n    (-0.809, 0.588),   # Vertex 3\n    (-0.809, -0.588),  # Vertex 4\n    (0.309, -0.951)    # Vertex 5\n]\n\n# Extract x and y coordinates\ny_coords = [vertex[0] for vertex in vertices]\nx_coords = [vertex[1] for vertex in vertices]\n\n# Plot the regular pentagon\nplt.figure(figsize=(10, 10))\nplt.plot(x_coords + [x_coords[0]], y_coords + [y_coords[0]], \'b-\')  # Closing the loop\nplt.scatter(x_coords[0], y_coords[0], s=200, color=\'dodgerblue\',alpha=0.5, label=\'Area A\')\nplt.scatter(x_coords[1], y_coords[1], s=200, color=\'indigo\',alpha=0.5, label=\'Area B\')\nplt.scatter(x_coords[2], y_coords[2], s=200, color=\'blue\',alpha=0.5, label=\'Area C\')\nplt.scatter(x_coords[3], y_coords[3], s=200, color=\'limegreen\',alpha=0.5, label=\'Area D\')\nplt.scatter(x_coords[4], y_coords[4], s=200, color=\'black\',alpha=0.

Vertex 1: (1, 0)
Vertex 2: (0.309, 0.951)
Vertex 3: (-0.809, 0.588)
Vertex 4: (-0.809, -0.588)
Vertex 5: (0.309, -0.951)

In [137]:
crossed_data.loc[crossed_data['Zone'] == 'A', 'y'] = 1
crossed_data.loc[crossed_data['Zone'] == 'B', 'y'] = 0.309
crossed_data.loc[crossed_data['Zone'] == 'C', 'y'] = -0.809
crossed_data.loc[crossed_data['Zone'] == 'D', 'y'] = -0.809
crossed_data.loc[crossed_data['Zone'] == 'E', 'y'] = 0.309

crossed_data.loc[crossed_data['Zone'] == 'A', 'x'] = 0
crossed_data.loc[crossed_data['Zone'] == 'B','x'] = 0.951
crossed_data.loc[crossed_data['Zone'] == 'C','x'] = 0.588
crossed_data.loc[crossed_data['Zone'] == 'D','x'] = -0.588
crossed_data.loc[crossed_data['Zone'] == 'E','x'] = -0.951

Simulation Map

In [138]:
from PIL import Image

img = Image.open('foo3.png')

fig = px.scatter(
            crossed_data,
            x="x",
            y="y",
            animation_frame="Date",
            animation_group="Container",
            #size="pop",
            color="Container",
            hover_name="Container",
            #size_max=55,
            range_x=[-1, 1],
            range_y=[-1.25, 1.25],
            width=600, height=600)

fig.update_layout(
    autosize=False,
    xaxis = dict(
        tickmode = 'array',
        tickvals = []
    ),
    yaxis = dict(
        tickmode = 'array',
        tickvals = []),
    yaxis_tickformat=' ')

fig.update_layout(showlegend=False)
fig.update_xaxes(visible=False)
fig.update_yaxes(visible=False)

fig.add_layout_image(
        dict(
            source=img,
            xref="x",
            yref="y",
            x=-1,
            y=1,
            sizex=2,
            sizey=2,
            sizing="stretch",
            opacity=1,
            layer="below")
)



fig.show()